In [1]:
from IPython.display import clear_output
import os

import numpy as np
import pandas as pd

from prereise.gather.demanddata.bldg_electrification import const
from prereise.gather.demanddata.bldg_electrification.helper import read_shapefile
from prereise.gather.demanddata.bldg_electrification.load_projection import (
    predict_scenario,
)
from prereise.gather.demanddata.bldg_electrification.load_projection_scenario import (
    LoadProjectionScenario,
)

In [3]:
geom_names = ["NYIS-ZONA", "TX"]
load_zones = []
for geom in geom_names:
    if geom in const.zone_names:
        load_zones.append(geom)
    elif geom in const.iso_zone_names.keys():
        load_zones.extend(const.iso_zone_names[geom])
    else:
        raise TypeError(
            f"Geometry name {geom} is invalid. It must be name of either a load zone,"
            "defined in list 'zone_names' in const.py, or a balancing authority,"
            "defined by the keys in dict 'iso_zone_names' in const.py"
        )

zone_df = pd.concat(
    list(
        pd.Series(data=load_zones).apply(
            lambda x: pd.read_csv(
                f"https://besciences.blob.core.windows.net/datasets/bldg_el/profile_stats/{x}_stats_{const.base_year}.csv",
                index_col=0,
            )
        )
    ),
    axis=1,
)  # user can get profile stats either by downloading prepared files from blob storage, or calling
# function 'zonal_data' in zone_profile_generator.py
zone_df.columns = load_zones
for clas in const.classes:
    zone_df.rename(
        {
            f"frac_elec_{clas}_hp": f"frac_hp_{clas}_heat",
        },
        inplace=True,
    )
    zone_df.loc[f"frac_resist_{clas}_heat", :] = (
        zone_df.loc[f"frac_elec_{clas}_heat", :]
        - zone_df.loc[f"frac_hp_{clas}_heat", :]
    )
zone_df.loc["frac_elec_com_cool", :] = zone_df.loc["frac_elec_res_cool", :]
zone_df.loc["year", :] = const.base_year
zone_df = zone_df.loc[
    [
        "year",
        "pop",
        "res_area_m2",
        "frac_hp_res_heat",
        "frac_resist_res_heat",
        "frac_ff_res_heat",
        "frac_elec_res_cool",
        "frac_elec_dhw_res",
        "frac_ff_dhw_res",
        "frac_elec_other_res",
        "frac_ff_other_res",
        "com_area_m2",
        "frac_hp_com_heat",
        "frac_resist_com_heat",
        "frac_ff_com_heat",
        "frac_elec_com_cool",
        "frac_elec_dhw_com",
        "frac_ff_dhw_com",
        "frac_elec_cook_com",
        "frac_ff_cook_com",
    ],
    :,
]
print("Estimation of base year building stock floor area and energy usages:")
display(zone_df)

Estimation of base year building stock floor area and energy usages:


,NYIS-ZONA,ERCO-C,ERCO-E,ERCO-FW,ERCO-N,ERCO-NC,ERCO-S,ERCO-SC,ERCO-W
year,2.019000e+03,2.019000e+03,2.019000e+03,2.019000e+03,2.019000e+03,2.019000e+03,2.019000e+03,2.019000e+03,2.019000e+03
pop,1.709888e+06,6.014404e+06,1.080635e+06,4.145662e+05,7.106414e+05,7.303966e+06,2.255179e+06,4.069627e+06,6.288678e+05
res_area_m2,1.238626e+08,4.387601e+08,8.138928e+07,3.061680e+07,5.717994e+07,5.420773e+08,1.364299e+08,3.047250e+08,5.010805e+07
frac_hp_res_heat,6.565478e-03,6.519546e-02,2.850208e-01,1.879996e-01,1.973527e-01,1.450226e-01,2.976339e-01,2.041959e-01,2.631244e-01
frac_resist_res_heat,9.089240e-02,4.076140e-01,3.053806e-01,2.759591e-01,2.662318e-01,4.098231e-01,4.614778e-01,3.531112e-01,3.044306e-01
frac_ff_res_heat,8.744296e-01,4.652416e-01,3.203000e-01,4.673519e-01,4.620187e-01,3.722410e-01,1.395176e-01,3.668084e-01,3.469191e-01
frac_elec_res_cool,7.207728e-01,9.999208e-01,9.995107e-01,9.992424e-01,9.964345e-01,9.996060e-01,9.999920e-01,9.998586e-01,9.994026e-01
frac_elec_dhw_res,3.533697e-01,6.303020e-01,7.342840e-01,6.224749e-01,6.221446e-01,7.028435e-01,8.834682e-01,7.050200e-01,7.140818e-01
frac_ff_dhw_res,7.665636e-01,4.088066e-01,2.704485e-01,4.008693e-01,3.948743e-01,3.251957e-01,1.163329e-01,3.179709e-01,2.952494e-01
frac_elec_other_res,3.959621e-01,6.948834e-01,7.807011e-01,6.884241e-01,6.881511e-01,7.547529e-01,9.038248e-01,7.565492e-01,7.640280e-01


In [4]:
zone_stats_proj = {}
for load_zone in load_zones:
    zone_stats_proj[load_zone] = pd.DataFrame({"yr2019": zone_df[load_zone]})
display(zone_stats_proj)

{'NYIS-ZONA':                             yr2019
 year                  2.019000e+03
 pop                   1.709888e+06
 res_area_m2           1.238626e+08
 frac_hp_res_heat      6.565478e-03
 frac_resist_res_heat  9.089240e-02
 frac_ff_res_heat      8.744296e-01
 frac_elec_res_cool    7.207728e-01
 frac_elec_dhw_res     3.533697e-01
 frac_ff_dhw_res       7.665636e-01
 frac_elec_other_res   3.959621e-01
 frac_ff_other_res     7.745634e-01
 com_area_m2           5.241914e+07
 frac_hp_com_heat      2.500068e-02
 frac_resist_com_heat  8.063455e-02
 frac_ff_com_heat      8.459865e-01
 frac_elec_com_cool    7.207728e-01
 frac_elec_dhw_com     3.775824e-01
 frac_ff_dhw_com       5.574923e-01
 frac_elec_cook_com    1.695459e-01
 frac_ff_cook_com      3.712183e-01,
 'ERCO-C':                             yr2019
 year                  2.019000e+03
 pop                   6.014404e+06
 res_area_m2           4.387601e+08
 frac_hp_res_heat      6.519546e-02
 frac_resist_res_heat  4.076140e-01
 fra

In [4]:
# provide scenario input zone-by-zone (run this cell and the one below. Another option provided is to run code in the cell twice below to provide input once for all target load zones)
building_type = {"res": "residential", "com": "commercial"}
for load_zone in load_zones:
    zone_stats = zone_stats_proj[load_zone]
    print(f"Scenario input for {load_zone}")
    # basic stats and building stock floor area
    zone_stats.loc["year", "sc1"] = int(input("target projection year: "))
    clear_output(wait=True)
    zone_stats.loc["pop_ann_grow_rate", "sc1"] = float(
        input("annual population growth rate: ")
    )
    clear_output(wait=True)
    zone_stats.loc["pop", "sc1"] = zone_stats.loc["pop", "yr2019"] * (
        (1 + zone_stats.loc["pop_ann_grow_rate", "sc1"])
        ** (zone_stats.loc["year", "sc1"] - zone_stats.loc["year", "yr2019"])
    )
    for clas in const.classes:
        floor_area_growth = input(
            f"{building_type[clas]} floor area:\nannual floor area growth rate, or enter 'pop' or leave blank to use the same growth "
            "rate as population growth, or enter '0' for no change in floor area compared to base year "
        )
        clear_output(wait=True)
        if floor_area_growth == "pop" or floor_area_growth == "":
            zone_stats.loc[f"{clas}_area_ann_grow_rate", "sc1"] = zone_stats.loc[
                "pop_ann_grow_rate", "sc1"
            ]
        elif floor_area_growth == "0":
            zone_stats.loc[f"{clas}_area_ann_grow_rate", "sc1"] = 0
        else:
            zone_stats.loc[f"{clas}_area_ann_grow_rate", "sc1"] = float(
                floor_area_growth
            )
        zone_stats.loc[f"{clas}_area_m2", "sc1"] = zone_stats.loc[
            f"{clas}_area_m2", "yr2019"
        ] * (
            (1 + zone_stats.loc[f"{clas}_area_ann_grow_rate", "sc1"])
            ** (zone_stats.loc["year", "sc1"] - zone_stats.loc["year", "yr2019"])
        )

        # heating
        heating_stats = input(
            f"{building_type[clas]} heating:\nenter fraction of {building_type[clas]} heating provided by heat pumps, resistive heater"
            ", and fossil fuel sequentially separated by ',', or enter blank to switch to provide fossil fuel electrification rate inputs "
        )
        clear_output(wait=True)
        if heating_stats == "":
            heating_electrify = input(
                "enter fraction of fossil fuel heating and resistance heating replaced by heat pump sequentially "
                "separated by ',', or enter blank for no change to the heating energy source types "
            )
            clear_output(wait=True)
            if heating_electrify == "":
                zone_stats.loc[
                    [
                        f"frac_hp_{clas}_heat",
                        f"frac_resist_{clas}_heat",
                        f"frac_ff_{clas}_heat",
                    ],
                    "sc1",
                ] = zone_stats.loc[
                    [
                        f"frac_hp_{clas}_heat",
                        f"frac_resist_{clas}_heat",
                        f"frac_ff_{clas}_heat",
                    ],
                    "yr2019",
                ]
            else:
                zone_stats.loc[f"ff2hp_{clas}", "sc1"] = list(
                    map(float, heating_electrify.split(","))
                )[0]
                zone_stats.loc[f"resist2hp_{clas}", "sc1"] = list(
                    map(float, heating_electrify.split(","))
                )[1]
        else:
            zone_stats.loc[
                [
                    f"frac_hp_{clas}_heat",
                    f"frac_resist_{clas}_heat",
                    f"frac_ff_{clas}_heat",
                ],
                "sc1",
            ] = list(map(float, heating_stats.split(",")))

        # cooling
        cooling_input = input(
            f"{building_type[clas]} cooling:\nfraction of floor area has electric air conditioning, or enter blank for no change "
        )
        clear_output(wait=True)
        zone_stats.loc[f"frac_elec_{clas}_cool", "sc1"] = (
            zone_stats.loc[f"frac_elec_{clas}_cool", "yr2019"]
            if cooling_input == ""
            else float(cooling_input)
        )
        # dhw
        dhw_input = input(
            f"{building_type[clas]} dhw:\nfraction of floor area use fossil fuel for hot water generation, or enter blank for no change "
        )
        zone_stats.loc[f"frac_ff_dhw_{clas}", "sc1"] = (
            zone_stats.loc[f"frac_ff_dhw_{clas}", "yr2019"]
            if dhw_input == ""
            else float(dhw_input)
        )
        # cooking
        cook_other = "cook" if clas == "com" else "other"
        cook_input = input(
            f"{building_type[clas]} cooking:\nfraction of floor area use fossil fuel for cooking, or enter blank for no change "
        )
        zone_stats.loc[f"frac_ff_{cook_other}_{clas}", "sc1"] = (
            zone_stats.loc[f"frac_ff_{cook_other}_{clas}", "yr2019"]
            if dhw_input == ""
            else float(dhw_input)
        )


display(zone_stats_proj)

{'NYIS-ZONA':                               yr2019           sc1
 year                    2.019000e+03  2.040000e+03
 pop                     1.709888e+06  1.898694e+06
 res_area_m2             1.238626e+08  1.375395e+08
 frac_hp_res_heat        6.565478e-03  1.000000e+00
 frac_resist_res_heat    9.089240e-02  0.000000e+00
 frac_ff_res_heat        8.744296e-01  0.000000e+00
 frac_elec_res_cool      7.207728e-01  7.207728e-01
 frac_elec_dhw_res       3.533697e-01           NaN
 frac_ff_dhw_res         7.665636e-01  0.000000e+00
 frac_elec_other_res     3.959621e-01           NaN
 frac_ff_other_res       7.745634e-01  0.000000e+00
 com_area_m2             5.241914e+07  5.820727e+07
 frac_hp_com_heat        2.500068e-02           NaN
 frac_resist_com_heat    8.063455e-02           NaN
 frac_ff_com_heat        8.459865e-01           NaN
 frac_elec_com_cool      7.207728e-01  7.207728e-01
 frac_elec_dhw_com       3.775824e-01           NaN
 frac_ff_dhw_com         5.574923e-01  0.000000e+00

In [ ]:
# choose device efficiency model
for load_zone in load_zones:
    zone_stats = zone_stats_proj[load_zone]
    zone_stats.loc["heat_hp_type", "yr2019"] = "midperfhp"
    zone_stats.loc["heat_hp_type", "sc1"] = input(
        "energy efficiency model of air source heat pump, select from 'midperfhp' and 'advperfhp'.\nChoose 'advperfhp' if anticipate heat pump efficiency improve "
    )
    zone_stats.loc["dhw_hp_type", "sc1"] = input(
        "energy efficiency model of hot water heat pump, select from 'midperfhp' and 'advperfhp' "
    )
    zone_stats.loc["cook_eff", "sc1"] = input(
        "energy efficiency model of electric cooking, select from 'low' and 'high' "
    )
    zone_stats.loc["cool_energy_intensity(relative)", "yr2019"] = 1
    zone_stats.loc["cool_energy_intensity(relative)", "sc1"] = input(
        "energy efficiency of cooling, '1' represents current efficiency "
    )

In [5]:
# provide scenario input for all zone
building_type = {"res": "residential", "com": "commercial"}
year_input = int(input("target projection year: "))
clear_output(wait=True)
pop_growth_input = float(input("annual population growth rate: "))
clear_output(wait=True)
for clas in const.classes:
    floor_area_growth = input(
        f"{building_type[clas]} floor area:\nannual floor area growth rate, or enter 'pop' or leave blank to use the same growth "
        "rate as population growth, or enter '0' for no change in floor area compared to base year "
    )
    clear_output(wait=True)
    # heating
    heating_electrify = input(
        "enter fraction of fossil fuel heating and resistance heating replaced by heat pump sequentially "
        "separated by ',', or enter blank for no change to the heating energy source types "
    )
    clear_output(wait=True)
    # cooling
    cooling_input = input(
        f"{building_type[clas]} cooling:\nfraction of floor area has electric air conditioning, or enter blank for no change "
    )
    clear_output(wait=True)
    # dhw
    dhw_input = input(
        f"{building_type[clas]} dhw:\nfraction of floor area use fossil fuel for hot water generation, or enter blank for no change "
    )
    # cooking
    cook_other = "cook" if clas == "com" else "other"
    cook_input = input(
        f"{building_type[clas]} cooking:\nfraction of floor area use fossil fuel for cooking, or enter blank for no change "
    )


for load_zone in load_zones:
    zone_stats = zone_stats_proj[load_zone]

    zone_stats.loc["year", "sc1"] = year_input
    zone_stats.loc["pop_ann_grow_rate", "sc1"] = pop_growth_input

    zone_stats.loc["pop", "sc1"] = zone_stats.loc["pop", "yr2019"] * (
        (1 + zone_stats.loc["pop_ann_grow_rate", "sc1"])
        ** (zone_stats.loc["year", "sc1"] - zone_stats.loc["year", "yr2019"])
    )

    for clas in const.classes:
        if floor_area_growth == "pop" or floor_area_growth == "":
            zone_stats.loc[f"{clas}_area_ann_grow_rate", "sc1"] = zone_stats.loc[
                "pop_ann_grow_rate", "sc1"
            ]
        elif floor_area_growth == "0":
            zone_stats.loc[f"{clas}_area_ann_grow_rate", "sc1"] = 0
        else:
            zone_stats.loc[f"{clas}_area_ann_grow_rate", "sc1"] = float(
                floor_area_growth
            )
        zone_stats.loc[f"{clas}_area_m2", "sc1"] = zone_stats.loc[
            f"{clas}_area_m2", "yr2019"
        ] * (
            (1 + zone_stats.loc[f"{clas}_area_ann_grow_rate", "sc1"])
            ** (zone_stats.loc["year", "sc1"] - zone_stats.loc["year", "yr2019"])
        )

        if heating_electrify == "":
            zone_stats.loc[
                [
                    f"frac_hp_{clas}_heat",
                    f"frac_resist_{clas}_heat",
                    f"frac_ff_{clas}_heat",
                ],
                "sc1",
            ] = zone_stats.loc[
                [
                    f"frac_hp_{clas}_heat",
                    f"frac_resist_{clas}_heat",
                    f"frac_ff_{clas}_heat",
                ],
                "yr2019",
            ]
        else:
            zone_stats.loc[f"ff2hp_{clas}", "sc1"] = list(
                map(float, heating_electrify.split(","))
            )[0]
            zone_stats.loc[f"resist2hp_{clas}", "sc1"] = list(
                map(float, heating_electrify.split(","))
            )[1]

        zone_stats.loc[f"frac_elec_{clas}_cool", "sc1"] = (
            zone_stats.loc[f"frac_elec_{clas}_cool", "yr2019"]
            if cooling_input == ""
            else float(cooling_input)
        )

        zone_stats.loc[f"frac_ff_dhw_{clas}", "sc1"] = (
            zone_stats.loc[f"frac_ff_dhw_{clas}", "yr2019"]
            if dhw_input == ""
            else float(dhw_input)
        )
        cook_other = "cook" if clas == "com" else "other"
        zone_stats.loc[f"frac_ff_{cook_other}_{clas}", "sc1"] = (
            zone_stats.loc[f"frac_ff_{cook_other}_{clas}", "yr2019"]
            if dhw_input == ""
            else float(dhw_input)
        )


display(zone_stats_proj)

{'NYIS-ZONA':                               yr2019           sc1
 year                    2.019000e+03  2.040000e+03
 pop                     1.709888e+06  1.898694e+06
 res_area_m2             1.238626e+08  1.375395e+08
 frac_hp_res_heat        6.565478e-03           NaN
 frac_resist_res_heat    9.089240e-02           NaN
 frac_ff_res_heat        8.744296e-01           NaN
 frac_elec_res_cool      7.207728e-01  7.207728e-01
 frac_elec_dhw_res       3.533697e-01           NaN
 frac_ff_dhw_res         7.665636e-01  0.000000e+00
 frac_elec_other_res     3.959621e-01           NaN
 frac_ff_other_res       7.745634e-01           NaN
 com_area_m2             5.241914e+07  5.820727e+07
 frac_hp_com_heat        2.500068e-02           NaN
 frac_resist_com_heat    8.063455e-02           NaN
 frac_ff_com_heat        8.459865e-01           NaN
 frac_elec_com_cool      7.207728e-01  7.207728e-01
 frac_elec_dhw_com       3.775824e-01           NaN
 frac_ff_dhw_com         5.574923e-01  0.000000e+00

In [6]:
# choose device efficiency model
heat_hp_type = input(
    "energy efficiency model of air source heat pump, select from 'midperfhp' and 'advperfhp'.\nChoose 'advperfhp' if anticipate heat pump efficiency improve "
)
dhw_hp_type = input(
    "energy efficiency model of hot water heat pump, select from 'midperfhp' and 'advperfhp' "
)
cook_eff = input(
    "energy efficiency model of electric cooking, select from 'low' and 'high' "
)
cool_energy_intensity = input(
    "energy efficiency of cooling, '1' represents current efficiency "
)

for load_zone in load_zones:
    zone_stats = zone_stats_proj[load_zone]
    zone_stats.loc["heat_hp_type", "yr2019"] = "midperfhp"
    zone_stats.loc["heat_hp_type", "sc1"] = heat_hp_type
    zone_stats.loc["dhw_hp_type", "sc1"] = dhw_hp_type
    zone_stats.loc["cook_eff", "sc1"] = cook_eff
    zone_stats.loc["cool_energy_intensity(relative)", "yr2019"] = 1
    zone_stats.loc["cool_energy_intensity(relative)", "sc1"] = cool_energy_intensity

In [7]:
# save scenario input for later reference
for load_zone in load_zones:
    zone_stats_proj[load_zone].to_csv(
        os.path.join(
            os.getcwd(), "projection", "scenario_inputs", f"{load_zone}_stats.csv"
        )
    )

In [8]:
# run load_projection.py

# Weather year to produce load profiles. If multiple years,
# then time series result will show for more than one year
weather_years = [2018, 2019]

# new heat pump load profile assumption. User can select whether to use electric profile or fossil fuel profile to estimate
# electrified fossil fuel consumption for heating
new_hp_profile = "elec"  # "elec" or "ff"

zone_names = load_zones
zone_name_shps = list(
    map(lambda x: const.zone_name_shps[const.zone_names.index(x)], load_zones)
)

os.makedirs(
    os.path.join(os.getcwd(), "Profiles"),
    exist_ok=True,
)
os.makedirs(
    os.path.join(os.getcwd(), "projection", "results"),
    exist_ok=True,
)

for i in range(len(zone_names)):
    zone_name, zone_name_shp = zone_names[i], zone_name_shps[i]
    scen_data = zone_stats_proj[load_zone].copy()
    # print(scen_data)
    base_scenarios = LoadProjectionScenario("base", scen_data.pop("yr2019"))
    print(f"base scenario: year {const.base_year}, weather year: {weather_years}")

    proj_scenarios = {}
    for name, values in scen_data.iteritems():
        proj_scenarios[name] = LoadProjectionScenario(name, values, base_scenarios)
        print(f"projection scenario {name}, year {proj_scenarios[name].year}")

    zone_profile_load_mwh = predict_scenario(
        zone_name,
        zone_name_shp,
        base_scenarios,
        proj_scenarios,
        weather_years,
        new_hp_profile,
    )

    for name, values in zone_profile_load_mwh.items():
        zone_profile_load_mwh[name].to_csv(
            os.path.join(
                os.getcwd(),
                "projection",
                "results",
                f"{zone_name}_{name}_mwh.csv",
            )
        )

base scenario: year 2019, weather year: [2018, 2019]
projection scenario sc1, year 2040
base scenario: year 2019, weather year: [2018, 2019]
projection scenario sc1, year 2040
generating hot water profiles for TX...
generating hot water profiles for TX...
generating hot water profiles for TX...
generating hot water profiles for TX...
generating ff cooking profiles for TX...
base scenario: year 2019, weather year: [2018, 2019]
projection scenario sc1, year 2040
base scenario: year 2019, weather year: [2018, 2019]
projection scenario sc1, year 2040
generating hot water profiles for NM...
generating hot water profiles for NM...
generating hot water profiles for NM...
generating hot water profiles for NM...
generating ff cooking profiles for NM...
base scenario: year 2019, weather year: [2018, 2019]
projection scenario sc1, year 2040
generating hot water profiles for OK...
generating hot water profiles for OK...
generating hot water profiles for OK...
generating hot water profiles for OK..